In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import time
import re
import urllib
import pandas as pd
import seaborn as sns
from requests.exceptions import HTTPError

In [2]:
player_url = 'https://www76.myfantasyleague.com/2018/export?TYPE=players&DETAILS=&SINCE=&PLAYERS=&JSON=0'
request = urllib.request.urlopen(player_url)
request_data = request.readlines()
line = "".join(str(request_data))
player_id = re.findall(r"\d+", line)
player_id = player_id[405:]
player_pos = re.findall(r'position.\".+?\"', line)
player_pos = player_pos[384:]
player_pos = re.findall(r'\".+?\"', str(player_pos))
player_pos = [pos[1:3] for pos in player_pos]
player_dict = {"id" : player_id, "pos" : player_pos}
player_df = pd.DataFrame(data=player_dict)
player_df = player_df.set_index("id")

In [3]:
def id_to_pos(pid):
    if pid not in player_df.index:
        return "NaN"
    return player_df.loc[pid][0]

In [4]:
def most_popular(series):
    return series.iloc[0]

In [5]:
ultimate_data = pd.DataFrame()

In [6]:
def prepare_data(data, league_id, year, league):
    line = "".join(str(data))
    draft_results = re.findall(r"draftPick.+?player.\"\d+\"", line)
    if len(draft_results) == 0:
        return ""
    round_list = re.findall(r"\d+", str(draft_results))
    data = list(zip(*[iter(round_list)]*4))
    data = pd.DataFrame(data, columns=['Round', 'Pick', 'Franchise', 'Player'])
    data = pd.pivot_table(data, index='Franchise', columns='Round', values='Player', aggfunc = most_popular)
    data = data.iloc[:, 0:6]
    if data.isnull().values.any() or data.iloc[0][1] == '0000':
        return ""
    #converting player ids to position
    for col in data.iloc[:, 0:6].columns:
        data[col] = [id_to_pos(pid) for pid in data[col]]
    standings = get_league_results(year, league)
    data = data.join(standings)
    data["League"] = league_id
    data.index = data.index + league_id
    return data

In [7]:
def get_league_results(year, league):
    standings_df = pd.DataFrame()
    url = 'http://www56.myfantasyleague.com/{}/export?TYPE=leagueStandings&L={}'.format(year, league)
    request = urllib.request.urlopen(url)
    request_data = request.readlines()
    line = "".join(str(request_data))
    standings = re.findall(r"franchise id.+?pf=.+?\"", line)
    for team in standings:
        fid = team[14:18]
        pf = re.findall('\d+', str(team[71:]))
        column = [[fid, pf[0]]]
        standings_df = standings_df.append(column)
    standings_df = standings_df.set_index(0)
    standings_df = standings_df.rename(columns = {1 : "Points"})
    #Convert to standard units
    points = standings_df["Points"]
    points = list(map(int, points))
    standings_df["Points"] = points
    standings_df["Points"] = (standings_df["Points"] - np.average(points)) / np.std(points)
    return standings_df

In [8]:
years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
league = 10002
for i in np.arange(300):
    for year in years:
        league_num = str(year) + str(league)
        print(str(league) + ", " + str(year))
        try:
            url = 'http://www66.myfantasyleague.com/{}/export?TYPE=draftResults&L={}'.format(year, league)
            request = urllib.request.urlopen(url)
            request_data = request.readlines()
            try:
                new_draft = prepare_data(request_data, league_num, year, league)
            except:
                new_draft = "0"
            if len(new_draft) < 2:
                print("invalid league")
                continue
            else:
                print("valid league")
                ultimate_data = ultimate_data.append(new_draft)
                print(len(ultimate_data))
        except urllib.error.HTTPError:
            print("League does not exist")
    league += i

10002, 2008
invalid league
10002, 2009
invalid league
10002, 2010
invalid league
10002, 2011
invalid league
10002, 2012
invalid league
10002, 2013
valid league
12
10002, 2014
invalid league
10002, 2015
invalid league
10002, 2016
League does not exist
10002, 2017
invalid league
10002, 2008
invalid league
10002, 2009
invalid league
10002, 2010
invalid league
10002, 2011
invalid league
10002, 2012
invalid league
10002, 2013
valid league
24
10002, 2014
invalid league
10002, 2015
invalid league
10002, 2016
League does not exist
10002, 2017
invalid league
10003, 2008
League does not exist
10003, 2009
invalid league
10003, 2010
League does not exist
10003, 2011
invalid league
10003, 2012
League does not exist
10003, 2013
valid league
36
10003, 2014
invalid league
10003, 2015
League does not exist
10003, 2016
invalid league
10003, 2017
invalid league
10005, 2008
League does not exist
10005, 2009
invalid league
10005, 2010
valid league
52
10005, 2011
valid league
54
10005, 2012
valid league
66


valid league
628
10353, 2014
invalid league
10353, 2015
invalid league
10353, 2016
invalid league
10353, 2017
invalid league
10380, 2008
League does not exist
10380, 2009
invalid league
10380, 2010
invalid league
10380, 2011
League does not exist
10380, 2012
valid league
640
10380, 2013
League does not exist
10380, 2014
invalid league
10380, 2015
valid league
650
10380, 2016
League does not exist
10380, 2017
League does not exist
10408, 2008
League does not exist
10408, 2009
League does not exist
10408, 2010
valid league
662
10408, 2011
valid league
672
10408, 2012
invalid league
10408, 2013
valid league
684
10408, 2014
League does not exist
10408, 2015
invalid league
10408, 2016
invalid league
10408, 2017
League does not exist
10437, 2008
invalid league
10437, 2009
League does not exist
10437, 2010
invalid league
10437, 2011
League does not exist
10437, 2012
League does not exist
10437, 2013
invalid league
10437, 2014
League does not exist
10437, 2015
invalid league
10437, 2016
invali

League does not exist
11487, 2009
invalid league
11487, 2010
valid league
1083
11487, 2011
invalid league
11487, 2012
invalid league
11487, 2013
invalid league
11487, 2014
League does not exist
11487, 2015
invalid league
11487, 2016
valid league
1095
11487, 2017
valid league
1107
11542, 2008
League does not exist
11542, 2009
League does not exist
11542, 2010
invalid league
11542, 2011
invalid league
11542, 2012
invalid league
11542, 2013
invalid league
11542, 2014
valid league
1119
11542, 2015
valid league
1139
11542, 2016
valid league
1149
11542, 2017
valid league
1159
11598, 2008
invalid league
11598, 2009
invalid league
11598, 2010
invalid league
11598, 2011
valid league
1171
11598, 2012
valid league
1187
11598, 2013
League does not exist
11598, 2014
valid league
1201
11598, 2015
valid league
1213
11598, 2016
invalid league
11598, 2017
invalid league
11655, 2008
League does not exist
11655, 2009
valid league
1225
11655, 2010
invalid league
11655, 2011
League does not exist
11655, 20

invalid league
13323, 2013
valid league
1637
13323, 2014
invalid league
13323, 2015
valid league
1649
13323, 2016
League does not exist
13323, 2017
League does not exist
13405, 2008
invalid league
13405, 2009
League does not exist
13405, 2010
invalid league
13405, 2011
valid league
1659
13405, 2012
invalid league
13405, 2013
valid league
1673
13405, 2014
League does not exist
13405, 2015
League does not exist
13405, 2016
valid league
1689
13405, 2017
valid league
1705
13488, 2008
League does not exist
13488, 2009
invalid league
13488, 2010
invalid league
13488, 2011
League does not exist
13488, 2012
invalid league
13488, 2013
valid league
1721
13488, 2014
invalid league
13488, 2015
invalid league
13488, 2016
valid league
1733
13488, 2017
valid league
1745
13572, 2008
valid league
1757
13572, 2009
League does not exist
13572, 2010
invalid league
13572, 2011
invalid league
13572, 2012
valid league
1765
13572, 2013
invalid league
13572, 2014
invalid league
13572, 2015
invalid league
13572

League does not exist
15997, 2008
League does not exist
15997, 2009
invalid league
15997, 2010
League does not exist
15997, 2011
invalid league
15997, 2012
valid league
2157
15997, 2013
League does not exist
15997, 2014
invalid league
15997, 2015
League does not exist
15997, 2016
invalid league
15997, 2017
invalid league
16107, 2008
League does not exist
16107, 2009
League does not exist
16107, 2010
valid league
2169
16107, 2011
League does not exist
16107, 2012
League does not exist
16107, 2013
League does not exist
16107, 2014
invalid league
16107, 2015
League does not exist
16107, 2016
League does not exist
16107, 2017
League does not exist
16218, 2008
League does not exist
16218, 2009
invalid league
16218, 2010
League does not exist
16218, 2011
invalid league
16218, 2012
invalid league
16218, 2013
invalid league
16218, 2014
invalid league
16218, 2015
League does not exist
16218, 2016
invalid league
16218, 2017
invalid league
16330, 2008
valid league
2179
16330, 2009
League does not

...

League does not exist
23368, 2013
League does not exist
23368, 2014
invalid league
23368, 2015
invalid league
23368, 2016
invalid league
23368, 2017
invalid league
23532, 2008
League does not exist
23532, 2009
invalid league
23532, 2010
League does not exist
23532, 2011
League does not exist
23532, 2012
invalid league
23532, 2013
invalid league
23532, 2014
invalid league
23532, 2015
invalid league
23532, 2016
invalid league
23532, 2017
invalid league
23697, 2008
League does not exist
23697, 2009
invalid league
23697, 2010
invalid league
23697, 2011
invalid league
23697, 2012
invalid league
23697, 2013
invalid league
23697, 2014
League does not exist
23697, 2015
valid league
3163
23697, 2016
invalid league
23697, 2017
invalid league
23863, 2008
invalid league
23863, 2009
League does not exist
23863, 2010
League does not exist
23863, 2011
valid league
3173
23863, 2012
valid league
3185
23863, 2013
League does not exist
23863, 2014
League does not exist
23863, 2015
invalid league
23863, 2

invalid league
28338, 2009
League does not exist
28338, 2010
League does not exist
28338, 2011
invalid league
28338, 2012
invalid league
28338, 2013
invalid league
28338, 2014
League does not exist
28338, 2015
invalid league
28338, 2016
invalid league
28338, 2017
League does not exist
28530, 2008
invalid league
28530, 2009
invalid league
28530, 2010
invalid league
28530, 2011
invalid league
28530, 2012
invalid league
28530, 2013
valid league
3656
28530, 2014
invalid league
28530, 2015
invalid league
28530, 2016
invalid league
28530, 2017
invalid league
28723, 2008
League does not exist
28723, 2009
League does not exist
28723, 2010
League does not exist
28723, 2011
League does not exist
28723, 2012
League does not exist
28723, 2013
invalid league
28723, 2014
invalid league
28723, 2015
valid league
3668
28723, 2016
League does not exist
28723, 2017
League does not exist
28917, 2008
League does not exist
28917, 2009
League does not exist
28917, 2010
League does not exist
28917, 2011
Leagu

valid league
4198
33873, 2013
League does not exist
33873, 2014
League does not exist
33873, 2015
invalid league
33873, 2016
invalid league
33873, 2017
invalid league
34092, 2008
League does not exist
34092, 2009
valid league
4206
34092, 2010
invalid league
34092, 2011
League does not exist
34092, 2012
invalid league
34092, 2013
invalid league
34092, 2014
invalid league
34092, 2015
invalid league
34092, 2016
League does not exist
34092, 2017
invalid league
34312, 2008
invalid league
34312, 2009
League does not exist
34312, 2010
invalid league
34312, 2011
invalid league
34312, 2012
League does not exist
34312, 2013
invalid league
34312, 2014
invalid league
34312, 2015
valid league
4218
34312, 2016
invalid league
34312, 2017
invalid league
34533, 2008
invalid league
34533, 2009
League does not exist
34533, 2010
valid league
4230
34533, 2011
League does not exist
34533, 2012
invalid league
34533, 2013
invalid league
34533, 2014
valid league
4240
34533, 2015
valid league
4250
34533, 2016
i

valid league
4752
40383, 2010
invalid league
40383, 2011
valid league
4764
40383, 2012
invalid league
40383, 2013
valid league
4784
40383, 2014
invalid league
40383, 2015
invalid league
40383, 2016
invalid league
40383, 2017
League does not exist
40630, 2008
invalid league
40630, 2009
League does not exist
40630, 2010
invalid league
40630, 2011
invalid league
40630, 2012
League does not exist
40630, 2013
invalid league
40630, 2014
invalid league
40630, 2015
League does not exist
40630, 2016
valid league
4794
40630, 2017
valid league
4804
40878, 2008
League does not exist
40878, 2009
invalid league
40878, 2010
League does not exist
40878, 2011
invalid league
40878, 2012
invalid league
40878, 2013
League does not exist
40878, 2014
League does not exist
40878, 2015
invalid league
40878, 2016
League does not exist
40878, 2017
League does not exist
41127, 2008
League does not exist
41127, 2009
invalid league
41127, 2010
invalid league
41127, 2011
valid league
4812
41127, 2012
invalid league

invalid league
47403, 2011
valid league
5279
47403, 2012
League does not exist
47403, 2013
League does not exist
47403, 2014
invalid league
47403, 2015
invalid league
47403, 2016
invalid league
47403, 2017
League does not exist
47677, 2008
League does not exist
47677, 2009
valid league
5289
47677, 2010
invalid league
47677, 2011
League does not exist
47677, 2012
League does not exist
47677, 2013
League does not exist
47677, 2014
invalid league
47677, 2015
valid league
5301
47677, 2016
invalid league
47677, 2017
invalid league
47952, 2008
League does not exist
47952, 2009
invalid league
47952, 2010
invalid league
47952, 2011
valid league
5313
47952, 2012
invalid league
47952, 2013
invalid league
47952, 2014
invalid league
47952, 2015
invalid league
47952, 2016
invalid league
47952, 2017
invalid league
48228, 2008
League does not exist
48228, 2009
invalid league
48228, 2010
League does not exist
48228, 2011
invalid league
48228, 2012
League does not exist
48228, 2013
League does not exis

In [9]:
ultimate_data.to_csv('./draftsamples.csv', index=True)

In [63]:
ultimate_data = pd.read_csv('draftsamples.csv')
ultimate_data = ultimate_data.rename(columns={"Unnamed: 0" : "TeamID"})
ultimate_data = ultimate_data.set_index("TeamID")
ultimate_data = ultimate_data.drop("07", axis=1)
ultimate_data

,01,02,03,04,05,06,League,Points
TeamID,,,,,,,,
1201310002,RB,QB,NaN,RB,NaN,NaN,201310002,0.514419
2201310002,NaN,WR,RB,WR,WR,RB,201310002,-0.499594
3201310002,NaN,NaN,WR,TE,QB,WR,201310002,-1.584765
4201310002,RB,RB,WR,NaN,NaN,RB,201310002,-0.410645
5201310002,RB,WR,QB,WR,NaN,RB,201310002,0.665631
6201310002,RB,WR,NaN,WR,QB,NaN,201310002,0.825738
7201310002,NaN,RB,WR,WR,TE,NaN,201310002,0.941371
8201310002,NaN,WR,RB,QB,WR,RB,201310002,0.318732
9201310002,RB,WR,NaN,RB,NaN,NaN,201310002,1.297165


In [64]:
ultimate_data = ultimate_data.replace('NaN', np.nan).dropna()

In [65]:
ultimate_data = ultimate_data.drop("League", axis=1)
ultimate_data

,01,02,03,04,05,06,Points
TeamID,,,,,,,
1201310003,RB,TE,RB,RB,QB,WR,0.658787
6201310003,RB,WR,RB,RB,QB,WR,-0.160545
3201010005,QB,RB,RB,WR,WR,WR,-1.264911
10201210005,RB,QB,WR,WR,TE,RB,0.325290
7201310005,RB,TE,WR,WR,WR,QB,-1.017693
9201310005,RB,RB,RB,WR,RB,QB,-1.017693
3201510005,RB,WR,QB,RB,WR,TE,-0.465549
4201510005,RB,WR,TE,QB,RB,WR,-0.465549
7201510005,RB,RB,WR,WR,QB,RB,-0.465549


In [66]:
ultimate_data = ultimate_data.replace(['DT', 'DE', 'LB', 'S"', 'PK'], np.nan)
ultimate_data = ultimate_data.dropna()
ultimate_data

,01,02,03,04,05,06,Points
TeamID,,,,,,,
1201310003,RB,TE,RB,RB,QB,WR,0.658787
6201310003,RB,WR,RB,RB,QB,WR,-0.160545
3201010005,QB,RB,RB,WR,WR,WR,-1.264911
10201210005,RB,QB,WR,WR,TE,RB,0.325290
7201310005,RB,TE,WR,WR,WR,QB,-1.017693
9201310005,RB,RB,RB,WR,RB,QB,-1.017693
3201510005,RB,WR,QB,RB,WR,TE,-0.465549
4201510005,RB,WR,TE,QB,RB,WR,-0.465549
7201510005,RB,RB,WR,WR,QB,RB,-0.465549


In [80]:
two_rounds = ultimate_data.groupby(["01", "02"]).agg(['mean', 'count'])
two_rounds = two_rounds[two_rounds["Points", "count"] > 20]
two_rounds

Points      
           mean count
01 02                
QB RB -0.115459    82
   WR -0.213690    56
RB QB -0.207175    84
   RB  0.152498   227
   TE  0.199599    30
   WR  0.002790   377
TE RB  0.510322    26
   WR  0.022784    21
WR QB  0.136605    49
   RB -0.020715   289
   TE -0.180670    37
   WR -0.017092   239

In [81]:
three_rounds = ultimate_data.groupby(["01", "02", "03"]).agg(['mean', 'count'])
three_rounds = three_rounds[three_rounds["Points", "count"] > 20]
three_rounds

Points      
              mean count
01 02 03                
QB RB RB  0.040181    22
      WR -0.230453    53
   WR RB -0.310939    33
RB QB RB -0.013548    25
      WR -0.304352    50
   RB QB -0.106246    35
      RB  0.304566    36
      WR  0.107799   144
   WR QB -0.150773    48
      RB  0.009576   142
      TE -0.007259    22
      WR  0.042963   165
WR QB RB  0.128569    28
   RB QB -0.087163    37
      RB  0.091438   102
      TE -0.234253    32
      WR -0.038917   118
   WR QB  0.163438    23
      RB -0.065539   118
      WR -0.069980    82

In [82]:
four_rounds = ultimate_data.groupby(["01", "02", "03", "04"]).agg(['mean', 'count'])
four_rounds = four_rounds[four_rounds["Points", "count"] > 20]
four_rounds

Points      
                 mean count
01 02 03 04                
QB RB WR RB -0.317397    22
RB QB WR WR -0.396939    25
   RB QB WR -0.129368    24
      RB WR  0.330758    24
      WR QB  0.167965    26
         RB -0.162110    26
         WR  0.170356    74
   WR QB RB -0.165154    24
      RB RB  0.155904    36
         WR -0.035801    77
      WR QB -0.044202    30
         RB -0.035912    73
         WR  0.297015    47
WR RB QB RB -0.129098    21
      RB WR  0.159401    61
      WR RB -0.142969    48
         WR  0.001506    37
   WR RB RB -0.064166    39
         WR -0.005974    44
      WR RB -0.134301    33
         WR -0.100112    35

In [94]:
bootstrapped = ultimate_data.sample(n=1532, replace=True)
bootstrapped

,01,02,03,04,05,06,Points
TeamID,,,,,,,
8201747130,QB,RB,WR,WR,RB,WR,-0.039188
5201611178,WR,RB,TE,RB,RB,WR,-0.337606
8201311955,RB,WR,RB,WR,WR,TE,1.171782
9201738205,WR,WR,RB,WR,WR,QB,-0.278207
11201645247,WR,RB,TE,WR,RB,QB,-1.467057
7201644718,WR,QB,WR,RB,RB,WR,-1.254638
7201610017,WR,TE,WR,RB,WR,RB,-0.379192
2201635202,RB,RB,WR,QB,RB,WR,0.503978
3201611178,WR,WR,QB,RB,TE,RB,-0.337606


In [100]:
bootstrapped_two = bootstrapped.groupby(["01", "02"]).agg(['mean', 'count'])
bootstrapped_two = bootstrapped_two[bootstrapped_two["Points", "count"] > 10]
bootstrapped_two

Points      
           mean count
01 02                
QB RB  0.040236    96
   WR -0.208068    62
RB QB -0.127473    80
   RB  0.149159   225
   TE  0.038370    24
   WR -0.024660   390
TE RB  0.748007    19
   WR -0.233241    22
WR QB  0.235323    48
   RB -0.030707   279
   TE -0.109287    31
   WR  0.035174   238

In [101]:
bootstrapped_three = bootstrapped.groupby(["01", "02", "03"]).agg(['mean', 'count'])
bootstrapped_three = bootstrapped_three[bootstrapped_three["Points", "count"] > 10]
bootstrapped_three

Points      
              mean count
01 02 03                
QB RB RB  0.379906    28
      WR -0.094916    63
   WR RB -0.271587    36
      WR  0.027705    16
RB QB RB  0.021939    29
      WR -0.174268    40
   RB QB -0.249832    42
      RB  0.487720    31
      WR  0.145484   145
   TE WR  0.185618    12
   WR QB -0.252705    48
      RB  0.082053   138
      TE -0.129635    15
      WR -0.036330   189
TE RB WR  0.525123    11
WR QB RB  0.260617    33
      WR  0.276447    13
   RB QB  0.087117    40
      RB  0.002803   106
      TE -0.116497    27
      WR -0.086828   106
   TE RB -0.011241    19
      WR -0.199303    11
   WR QB  0.020057    32
      RB -0.149355   106
      TE  0.837046    17
      WR  0.112427    83

In [102]:
bootstrapped_four = bootstrapped.groupby(["01", "02", "03", "04"]).agg(['mean', 'count'])
bootstrapped_four = bootstrapped_four[bootstrapped_four["Points", "count"] > 10]
bootstrapped_four

Points      
                 mean count
01 02 03 04                
QB RB RB WR  0.253067    22
      WR RB -0.146241    26
         TE -0.214894    11
         WR -0.002321    23
   WR RB WR -0.138126    17
      WR RB -0.044447    15
RB QB RB WR  0.095859    13
      WR RB -0.353388    11
         WR -0.276151    18
   RB QB WR -0.322067    32
      RB WR  0.775856    23
      WR QB  0.476005    33
         RB -0.320938    25
         TE  0.278561    21
         WR  0.114557    66
   TE WR WR  0.238886    11
   WR QB RB -0.384972    23
         WR  0.000531    18
      RB QB  0.153894    24
         RB  0.141590    43
         WR  0.019103    65
      WR QB -0.108520    30
         RB -0.072962    87
         TE -0.535373    14
         WR  0.176417    58
WR QB RB RB  0.569730    14
         WR  0.214933    13
      WR RB -0.243809    11
   RB QB RB -0.148528    22
         WR  0.628969    13
      RB QB -0.328675    21
         RB  0.006960    20
         WR  0.092980    60
      TE WR -0.726287    11
      WR QB -0.143389    18
         RB -0.074932    42
         TE -0.198189    14
         WR -0.021905    32
   WR QB RB -0.069040    19
      RB QB -0.346018    16
         RB -0.344906    35
         TE  0.118469    13
         WR  0.005627    42
      WR QB  0.175446    22
         RB  0.270865    24
         WR -0.112085    34